In [1]:
!pip -q install langchain chromadb pypdf langchain_groq langchain_community PyPDF2 sentence_transformers langchain_pinecone langchain_huggingface langchain_google_genai langchain_openai pyngrok streamlit langchain-groq

In [2]:
import os
from langchain import hub
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import Pinecone
import google.generativeai as genai
from langchain.chains import ConversationalRetrievalChain
from pinecone import Pinecone as pc
import streamlit as st
from pinecone import PodSpec
from langchain_groq import ChatGroq
from google.colab import files
from PyPDF2 import PdfReader
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Upload PDF files
uploaded_files = files.upload()
if uploaded_files != {}:
  # Initialize the text splitter
  text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

  # Loop through each uploaded file and process them
  for file_name, file_data in uploaded_files.items():
      # Save the uploaded file
      with open(file_name, "wb") as f:
          f.write(file_data)

      # Load and extract content from the uploaded PDF using PyPDFLoader
      loader = PyPDFLoader(file_name)
      document = loader.load()  # This returns a list of documents

      # Split the document into chunks
      texts = text_splitter.split_documents(document)

      # Print the number of chunks created and preview the first chunk
      print(f"Created {len(texts)} chunks from {file_name}")


  # Set API keys as environment variables
  os.environ["GROQ_API_KEY"] = "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"
  os.environ["PINECONE_API_KEY"] = "pcsk_6NHYkW_Kizjq1TGJ75RP2afEqJnbAZMY3JpjEviaC81dUbfpjNNxFwERFeXtuoM4TBFEdP"

  ## Set up the language model
  GROQ_API_KEY = os.getenv("GROQ_API_KEY")
  llm = ChatGroq(temperature=0,model_name="llama3-70b-8192", api_key=GROQ_API_KEY)

    # Set up embeddings
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/msmarco-distilbert-base-v4")

  # Initialize Pinecone client

  pine_client= pc(
      api_key = os.getenv("PINECONE_API_KEY"))
  index_name = "project-pdf-vectorized"

  # First, check if the index already exists. If it doesn't, create a new one.
  if index_name not in pine_client.list_indexes().names():
      print("Creating index")
      pine_client.create_index(name=index_name,
                        metric="cosine",
                        dimension=768,
                        spec=PodSpec(
                          environment="gcp-starter",
                          pod_type="starter",
                          pods=1)
      )


  vectorstore = Pinecone.from_documents(texts,
                        embeddings, index_name=index_name)

  retriever = vectorstore.as_retriever(search_kwargs={"k":3})

  QA_chain = ConversationalRetrievalChain.from_llm(
      llm=llm,
      chain_type = "stuff",
      retriever = retriever
  )
  q = ""
  while q != "exit":
    q = input("Ask question:")
    a = QA_chain({"question":q,"chat_history":[]})
    print(a["answer"])
else:
  print("Please run again and upload a file")

Saving Cs5368_2024Fall_Project.pdf to Cs5368_2024Fall_Project (8).pdf
Created 3 chunks from Cs5368_2024Fall_Project (8).pdf
Ask question:hey
Hey! I'm here to help with any questions you might have based on the text you provided. What's on your mind?
Ask question:what are the functionality
Based on the provided context, it appears that the task is to build an AI chatbot using any technology/framework, and the responses must be presented in a Colab notebook. The references provided suggest that the chatbot is related to RAG (Retrieval-Augmented Generation) and LangChain.

However, the specific functionalities of the chatbot are not explicitly mentioned in the context. If I had to make an educated guess, I would say that the chatbot is expected to have basic conversational capabilities, such as:

* Responding to user input
* Generating human-like text
* Retrieving information from a knowledge base or database
* Engaging in a conversation

But I don't know the exact functionalities require